# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 10:08AM,254757,10,WT5512 Out,Methapharm-G,Released
1,Jan 11 2023 9:59AM,254756,21,638204,"NBTY Global, Inc.",Released
2,Jan 11 2023 9:48AM,254755,21,638234,"NBTY Global, Inc.",Released
3,Jan 11 2023 9:47AM,254754,21,638194,"NBTY Global, Inc.",Released
4,Jan 11 2023 9:46AM,254753,10,MSP216998,Methapharm-G,Released
5,Jan 11 2023 9:46AM,254753,10,MSP216998,Methapharm-G,Released
6,Jan 11 2023 9:36AM,254749,10,PRF-44162-BO1,Bio-PRF,Released
7,Jan 11 2023 9:36AM,254749,10,PRF-46128-BO,Bio-PRF,Released
8,Jan 11 2023 9:36AM,254749,10,PRF-46384,Bio-PRF,Released
9,Jan 11 2023 9:36AM,254749,10,PRF-46388,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,254753,Released,2
35,254754,Released,1
36,254755,Released,1
37,254756,Released,1
38,254757,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254753,NaN,NaN,2.0
254754,NaN,NaN,1.0
254755,NaN,NaN,1.0
254756,NaN,NaN,1.0
254757,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0.0,4.0,14.0
254655,0.0,1.0,0.0
254657,0.0,0.0,1.0
254668,0.0,1.0,0.0
254673,8.0,3.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0,4,14
254655,0,1,0
254657,0,0,1
254668,0,1,0
254673,8,3,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,0,4,14
1,254655,0,1,0
2,254657,0,0,1
3,254668,0,1,0
4,254673,8,3,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,,4,14
1,254655,,1,
2,254657,,,1
3,254668,,1,
4,254673,8,3,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc."
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc."
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc."
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G
6,Jan 11 2023 9:36AM,254749,10,Bio-PRF
20,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix
21,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc."
24,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.
25,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G,,,1
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",,,1
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",,,1
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",,,1
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G,,,2
5,Jan 11 2023 9:36AM,254749,10,Bio-PRF,,,14
6,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,,,1
7,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",,,3
8,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,,,1
9,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1,,
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1,,
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",1,,
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",1,,
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G,2,,
5,Jan 11 2023 9:36AM,254749,10,Bio-PRF,14,,
6,Jan 11 2023 9:35AM,254750,10,Yusen – 3D Matrix,1,,
7,Jan 11 2023 9:31AM,254748,10,"Citieffe, Inc.",3,,
8,Jan 11 2023 9:23AM,254747,16,TASA USA Inc.,1,,
9,Jan 11 2023 9:20AM,254746,10,ISDIN Corporation,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1,,
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1,,
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",1,,
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",1,,
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1.0,NaN,NaN
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 10:08AM,254757,10,Methapharm-G,1.0,0.0,0.0
1,Jan 11 2023 9:59AM,254756,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jan 11 2023 9:48AM,254755,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jan 11 2023 9:47AM,254754,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jan 11 2023 9:46AM,254753,10,Methapharm-G,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4839935,142.0,34.0,8.0
15,509348,15.0,4.0,0.0
16,1018936,3.0,1.0,0.0
19,1273573,42.0,3.0,0.0
21,1273577,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4839935,142.0,34.0,8.0
1,15,509348,15.0,4.0,0.0
2,16,1018936,3.0,1.0,0.0
3,19,1273573,42.0,3.0,0.0
4,21,1273577,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,142.0,34.0,8.0
1,15,15.0,4.0,0.0
2,16,3.0,1.0,0.0
3,19,42.0,3.0,0.0
4,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,15,Released,15.0
2,16,Released,3.0
3,19,Released,42.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,8.0,0.0,0.0,0.0,0.0
Executing,34.0,4.0,1.0,3.0,1.0
Released,142.0,15.0,3.0,42.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,34.0,4.0,1.0,3.0,1.0
2,Released,142.0,15.0,3.0,42.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,8.0,0.0,0.0,0.0,0.0
1,Executing,34.0,4.0,1.0,3.0,1.0
2,Released,142.0,15.0,3.0,42.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()